In [ ]:
import librosa
import numpy as np
from scipy.spatial.distance import cosine
from datetime import timedelta
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function to load audio file and extract features
def extract_features(audio_path, sr=22050, n_mfcc=13, hop_length=512):
    y, sr = librosa.load(audio_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    return mfccs, sr, hop_length

# Function to compute similarity between two feature sets
def compute_similarity(features1, features2):
    # Flatten the feature matrices
    features1_flat = features1.flatten()
    features2_flat = features2.flatten()

    # Compute cosine similarity
    similarity = 1 - cosine(features1_flat, features2_flat)
    return similarity

In [ ]:
# Function to find similar segments
def find_similar_segments(movie_features, trailer_features, threshold=0.8):
    num_trailer_frames = trailer_features.shape[1]
    similar_segments = []

    for i in range(movie_features.shape[1] - num_trailer_frames + 1):
        movie_segment = movie_features[:, i:i + num_trailer_frames]
        similarity = compute_similarity(movie_segment, trailer_features)

        if similarity >= threshold:
            similar_segments.append(i)

    return similar_segments

In [ ]:
# Function to group timestamps into intervals
def group_into_intervals(timestamps, sr, hop_length, gap_threshold=1.0):
    if not timestamps:
        return []

    intervals = []
    start = timestamps[0] * hop_length / sr
    end = start

    for i in range(1, len(timestamps)):
        current_time = timestamps[i] * hop_length / sr
        previous_time = timestamps[i - 1] * hop_length / sr

        if current_time - previous_time <= gap_threshold:
            end = current_time
        else:
            intervals.append((start, end))
            start = current_time
            end = start

    intervals.append((start, end))
    return intervals

# Function to convert seconds to hh:mm:ss format
def seconds_to_hms(seconds):
    return str(timedelta(seconds=int(seconds)))

In [ ]:
# Function to analyze and predict common intervals
def predict_common_intervals(movie_trailer_pairs, test_movie_audio_path, threshold=0.8, gap_threshold=1.0):
    all_intervals = []

    for movie_audio_path, trailer_audio_path in movie_trailer_pairs:
        # Extract features
        movie_features, sr, hop_length = extract_features(movie_audio_path)
        trailer_features, _, _ = extract_features(trailer_audio_path, sr=sr, hop_length=hop_length)

        # Find similar segments
        similar_segments = find_similar_segments(movie_features, trailer_features, threshold)

        # Group into intervals
        similar_intervals = group_into_intervals(similar_segments, sr, hop_length, gap_threshold)
        all_intervals.extend(similar_intervals)

    # Analyze intervals to find common patterns
    common_intervals = Counter([tuple(map(lambda x: round(x, 2), interval)) for interval in all_intervals])

    # Test the common patterns on the fifth movie
    test_movie_features, sr, hop_length = extract_features(test_movie_audio_path)
    predicted_intervals = []

    for interval, count in common_intervals.items():
        start_sec, end_sec = interval
        test_segment = test_movie_features[:, int(start_sec * sr / hop_length):int(end_sec * sr / hop_length)]
        if test_segment.shape[1] > 0:
            similarity = compute_similarity(test_segment, test_segment)  # Self-similarity
            if similarity >= threshold:
                predicted_intervals.append(interval)

    # Output the intervals in hh:mm:ss format
    predicted_intervals_hms = [(seconds_to_hms(start), seconds_to_hms(end)) for start, end in predicted_intervals]

    return predicted_intervals_hms

# Paths to the audio files for the 4 movies and their trailers
movie_trailer_pairs = [
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M2_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T2_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M3_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T3_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M4_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T4_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M5_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T5_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M6_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T6_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M7_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T7_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M8_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T8_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M9_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T9_Audio.mp3"),
    ("/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M10_Audio.mp3", "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/T10_Audio.mp3")
]

In [ ]:
# Path to the audio file for the 5th movie
test_movie_audio_path = "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M1_Audio.mp3"

# Predict common intervals for the 5th movie
predicted_intervals = predict_common_intervals(movie_trailer_pairs, test_movie_audio_path, threshold=0.93, gap_threshold=1.0)


In [ ]:
# Display predicted intervals
print("Predicted Intervals (start and end times in hh:mm:ss) for the movie:")
for start_hms, end_hms in predicted_intervals:
    print(f"Start: {start_hms}, End: {end_hms}")

Predicted Intervals (start and end times in hh:mm:ss) for the movie:
Start: 0:05:56, End: 0:05:56
Start: 0:08:32, End: 0:08:32
Start: 0:01:16, End: 0:01:19
Start: 0:01:58, End: 0:02:06
Start: 0:05:30, End: 0:05:36
Start: 0:06:22, End: 0:06:33
Start: 0:09:57, End: 0:10:02


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
from datetime import timedelta

# Function to convert hh:mm:ss to seconds
def hms_to_seconds(hms):
    h, m, s = map(int, hms.split(':'))
    return h * 3600 + m * 60 + s

In [ ]:
# Path to the movie file
movie_file_path = "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M1_623.mp4"

# Load the movie file
movie_clip = VideoFileClip(movie_file_path)

# Convert intervals to seconds
predicted_intervals_seconds = [(hms_to_seconds(start), hms_to_seconds(end)) for start, end in predicted_intervals]


In [ ]:
# Extract clips based on intervals
clips = [movie_clip.subclip(start, end) for start, end in predicted_intervals_seconds]

# Concatenate the clips to form the trailer
if clips:
    trailer_clip = concatenate_videoclips(clips)
    # Save the trailer clip
    trailer_clip.write_videofile("/content/drive/MyDrive/Capstone/AudioProcessed/Predicted_T1_Horror9.mp4", codec="libx264", fps=24)
else:
    print("No clips to concatenate. Please check the intervals.")

# Close the movie clip
movie_clip.close()



chunk:  91%|█████████ | 1742/1919 [1:06:04<00:00, 435.12it/s, now=None]

chunk:  91%|█████████ | 1742/1919 [1:06:04<00:00, 435.12it/s, now=None]

Moviepy - Building video /content/drive/MyDrive/Capstone/AudioProcessed/Predicted_T1_Horror9.mp4.
MoviePy - Writing audio in Predicted_T1_Horror9TEMP_MPY_wvf_snd.mp3


                                                                   

chunk:  91%|█████████ | 1742/1919 [1:06:06<00:00, 435.12it/s, now=None]

chunk:  91%|█████████ | 1742/1919 [1:06:06<00:00, 435.12it/s, now=None]

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Capstone/AudioProcessed/Predicted_T1_Horror9.mp4





chunk:  91%|█████████ | 1742/1919 [1:06:33<00:00, 435.12it/s, now=None]

chunk:  91%|█████████ | 1742/1919 [1:06:33<00:00, 435.12it/s, now=None]

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Capstone/AudioProcessed/Predicted_T1_Horror9.mp4


**VIDEO PROCESSING**

In [ ]:
!pip install torch torchvision torchaudio
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16965, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 16965 (delta 82), reused 100 (delta 49), pack-reused 16805 (from 1)
Receiving objects: 100% (16965/16965), 15.71 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (11615/11615), done.
/content/yolov5/yolov5/yolov5/yolov5/yolov5


In [ ]:
import torch
import cv2
import numpy as np
import librosa
from scipy.spatial.distance import cosine
from collections import Counter
from datetime import timedelta
from moviepy.editor import VideoFileClip, concatenate_videoclips

# Load the pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-27 Python-3.10.12 torch-2.4.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 173MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# Function to check if humans are detected in the frame using YOLOv5
def detect_humans(frame):
    results = model(frame)
    # Extract the class labels (YOLOv5 has "person" with class label 0)
    detected_classes = results.xyxy[0][:, -1].numpy()  # Last column contains class labels
    return 0 in detected_classes  # 0 is the class for 'person'

# Function to check if the frame is dark (useful for horror trailer)
def is_dark(frame, brightness_threshold=50):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray_frame)
    return avg_brightness < brightness_threshold

# Function to extract video segments based on audio-predicted intervals and video features
def extract_final_clips(video_path, intervals, human_detection=True, brightness_check=True):
    video_clip = VideoFileClip(video_path)
    final_clips = []

    for start, end in intervals:
        subclip = video_clip.subclip(start, end)
        frames = []
        include_clip = False

        # Process each frame in the subclip
        for frame in subclip.iter_frames():
            # Check for humans
            if human_detection and detect_humans(frame):
                include_clip = True

            # Check for darkness in horror genre
            if brightness_check and is_dark(frame):
                include_clip = True

            # If either condition is true, add the frame
            if include_clip:
                frames.append(frame)

        # If clip meets criteria, include in final trailer
        if include_clip:
            final_clips.append(subclip)

    return final_clips

In [ ]:
# Path to the audio file for the test movie (5th movie)
test_movie_audio_path = "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M1_Audio.mp3"
test_movie_video_path = "/content/drive/MyDrive/Capstone/AudioProcessing/Horror/M1_623.mp4"


# Process video based on predicted intervals and video analysis
final_clips = extract_final_clips(test_movie_video_path, predicted_intervals_seconds)

# Concatenate and save the final trailer
if final_clips:
    final_trailer = concatenate_videoclips(final_clips)
    final_trailer.write_videofile("/content/drive/MyDrive/Capstone/Final_Trailer_1.mp4", codec="libx264", fps=24)
else:
    print("No valid clips for final trailer generation.")

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with a

Moviepy - Building video /content/drive/MyDrive/Capstone/Final_Trailer_1.mp4.
MoviePy - Writing audio in Final_Trailer_1TEMP_MPY_wvf_snd.mp3


                                                                   

chunk:  91%|█████████ | 1742/1919 [1:10:40<00:00, 435.12it/s, now=None]

chunk:  91%|█████████ | 1742/1919 [1:10:40<00:00, 435.12it/s, now=None]

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Capstone/Final_Trailer_1.mp4





chunk:  91%|█████████ | 1742/1919 [1:11:10<00:00, 435.12it/s, now=None]

chunk:  91%|█████████ | 1742/1919 [1:11:10<00:00, 435.12it/s, now=None]

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Capstone/Final_Trailer_1.mp4
